In [1]:
import requests
from pprint import pprint as pp
import numpy as np
import pandas as pd

# Actions to get all the game ID urls for every season
A list of urls for every game from the years that are in the range entered.

In [9]:
url_list = []
for year in range(2009,2019):
    url = "http://www.nfl.com/feeds-rs/schedules/{}.json".format(year)
    game_id_url = "http://www.nfl.com/feeds-rs/scores/byGame/"
    response = requests.get(url)
    results = response.json()
    games = results['gameSchedules']
    game_ids = []
    game_ids = ["{}{}.json".format(game_id_url, item[-1]) for game in games for item in game.items() if item[0] == "gameId"]
    url_list.extend(game_ids)

# Actions to get info for game
These actions call the NFL API and parse the json request that is returned. <br>
The information is then loaded into a pandas DataFrame object and exported as a ".csv" file.

In [5]:
def parse_data(url):
    row = []
    g_response = requests.get(url)
    g_results = g_response.json()
    for sect in g_results.items():
        if sect[0] == 'gameSchedule':
            for item in sect[-1].items():
                if (item[0] == "visitorTeam"):
                    continue
                elif (item[0] == "homeTeam"):
                    for it in item[-1].items():
                        if it[0] == "teamId":
                            team_id = it[-1]
                elif (item[0] == "gameId"):
                    game_id = item[-1]
                elif (item[0] == "site"):
                    for it in item[-1].items():
                        row.append(it[-1])
                else:
                    row.append(item[-1])
        else:
            for item in sect[-1].items():
                if (item[0] == 'homeTeamScore') | (item[0] == 'visitorTeamScore'):
                    tot = item[-1].get('pointTotal')
                    row.append(tot)
        stat_url = "http://www.nfl.com/feeds-rs/teamGameStats/{}/{}.json".format(team_id, game_id)
    s_response = requests.get(stat_url)
    s_results = s_response.json()
    home_team_stat_line = s_results['teamStatDetail']
    visitor_team_stat_line = s_results['opponentStatDetail']
    teams_stats = [home_team_stat_line, visitor_team_stat_line]
    for team in teams_stats:
        for item in team.items():
            row.append(item[-1])
    return row

In [6]:
def get_columns(url):
    row = []
    g_response = requests.get(url)
    g_results = g_response.json()
    for sect in g_results.items():
        if sect[0] == 'gameSchedule':
            for item in sect[-1].items():
                if (item[0] == "visitorTeam"):
                    continue
                elif (item[0] == "homeTeam"):
                    for it in item[-1].items():
                        if it[0] == "teamId":
                            team_id = it[-1]
                elif (item[0] == "gameId"):
                    game_id = item[-1]
                elif (item[0] == "site"):
                    for it in item[-1].items():
                        row.append(it[0])
                else:
                    row.append(item[0])
        else:
            for item in sect[-1].items():
                if (item[0] == 'homeTeamScore') | (item[0] == 'visitorTeamScore'):
                    row.append(item[0])
        stat_url = "http://www.nfl.com/feeds-rs/teamGameStats/{}/{}.json".format(team_id, game_id)
    s_response = requests.get(stat_url)
    s_results = s_response.json()
    home_team_stat_line = s_results['teamStatDetail']
    visitor_team_stat_line = s_results['opponentStatDetail']
    teams_stats = [home_team_stat_line, visitor_team_stat_line]
    for team in teams_stats:
        for item in team.items():
            row.append(item[0])
    return row

In [ ]:
data = []
for l in url_list[1:]:
    try:
        row = parse_data(l)
        data.append(row)
    except:
        print("Skipped!")

In [65]:
url_for_columns = url_list[0]
all_stats = get_columns(url_for_columns)
all_columns = get_columns(url_for_columns)
game_columns = all_columns[:24]
stat_columns = all_columns[24:]
cols = stat_columns[:109]

home_columns = []
for col in cols:
    home_columns.append("H{}".format(col))

visitor_columns = []
for col in cols:
    visitor_columns.append("V{}".format(col))

final_columns = []
final_columns.extend(game_columns)
final_columns.extend(home_columns)
final_columns.extend(visitor_columns)

In [ ]:
df = pd.DataFrame(data)
df.columns = final_columns
df.to_csv("../data/interim/game_data.csv", index=False)